## Hypothesis Testing Tutorial

This notebook outlined the process we used for hypothesis testing between for the microtubule catastrophe experiment. Specifically, we compare the results for fluorescent labeled and non-fluorescent labeled tubulin.

In [26]:
%load_ext autoreload
%autoreload 2
import os, sys
# Using alias for packages can reduce having to write out the entire name every single time you call them
import pandas as pd
import holoviews as hv
import bokeh.io
import scipy
import numpy as np
import scipy.stats as st
import MCAT_pkg as mc 
from bokeh.layouts import row, column

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mc?

Type:        module
String form: <module 'MCAT_pkg' from 'c:\\users\\rashi jeeda\\documents\\caltech\\classes\\3-1-bebi103a\\mcat-main\\mcat-main\\mcat_pkg\\MCAT_pkg\\__init__.py'>
File:        c:\users\rashi jeeda\documents\caltech\classes\3-1-bebi103a\mcat-main\mcat-main\mcat_pkg\mcat_pkg\__init__.py
Docstring:  
The contents of this package allow users to reproduce our microtubule catastrophe analysis. It includes:
Our methods for parsing the raw data
Methods for performing exploratory analysis
Statistical analysis modules (bootstrapping, hypothesis testing, MLE analysis, and model assessment)


In [3]:
# Identify the location that the data file is in  
data = "data/gardner_time_to_catastrophe_dic_tidy.csv"

# Read in the file as df

df = pd.read_csv(data, comment = "#")

In [4]:
# parse out the labeled and not_labeled data using our function
labeled, not_labeled = mc.separate_categories(df)

We can use mc.categorical_plot to gain ECDFs for the two samples with confidence intervals, so that we can observe the amount of overlap between the two datasets.

In [36]:
# plot the ECDFs for each of them with confidence intervals generated by bootstrapping
p = mc.categorical_plot(df, "time to catastrophe (s)", "labeled", conf_int = True, palette = ["limegreen", "darkgray"])
p.background_fill_color = "#fafafa"
bokeh.io.show(p)


For more quantiative metrics, we can perform a hypothesis test using bootstrapping, as well as look at means and confidence intervals for each of the datasets.

In [22]:
# Generate samples (10,000)
bs_reps = mc.draw_bs_reps_test_stat(labeled, not_labeled, size=10000)

# Compute p-value
stat = st.ks_2samp(labeled, not_labeled)[0]
p_val = np.sum(bs_reps >= stat) / len(bs_reps)

print("p-value =", p_val)

KeyboardInterrupt: 

In [7]:
# generate bootstrap replicates for the mean of each data set, then take the mean and confidence intervals
bs_reps_mean_tubulin = mc.draw_bs_reps_mean(labeled, size=10000)
bs_reps_mean_no_tubulin = mc.draw_bs_reps_mean(not_labeled, size = 10000)

mean_tubulin = np.mean(bs_reps_mean_tubulin)
mean_no_tubulin = np.mean(bs_reps_mean_no_tubulin)

conf_int_tubulin = np.percentile(bs_reps_mean_tubulin, [2.5, 97.5])
conf_int_no_tubulin = np.percentile(bs_reps_mean_no_tubulin, [2.5, 97.5])

print('Mean time to catastrophe with labeled tubulin is {} with 95% conf int : {}'.format(mean_tubulin,
                                                                                         conf_int_tubulin))
print('Mean time to catastrophe with nonlabeled tubulin is {} with 95% conf int: {}'.format(mean_no_tubulin,
                                                                                           conf_int_no_tubulin))

Mean time to catastrophe with labeled tubulin is 440.89370616113746 with 95% conf int : [401.39810427 481.56694313]
Mean time to catastrophe with nonlabeled tubulin is 412.2302947368421 with 95% conf int: [353.41842105 475.68421053]


We use the DKW inequality to further compare the two models as well as to observe the bounds for the confidence intervals.

In [31]:
mc.plot_conf_int??

Signature: mc.plot_conf_int(data, title, xlabel, color='tomato')
Source:   
def plot_conf_int(data, title, xlabel, color = 'tomato'):
    ''' plots an ECDF with confidence intervals 
    data : array
    contains raw data points from experiment
    title : string
    title for output graph
    xlabel : x axis label for output graph
    color : string (optional)
    if given, color for upper and lower confidence interval bounds
    
    Returns
    ________
    output : bokeh figure 
    '''
    x = np.linspace(0, max(data), 100)
    epsilon = np.sqrt(np.log(2/0.05)/(2 * len(data)))
    p = bokeh.plotting.figure(title=title, x_axis_label=xlabel, 
                   y_axis_label='ECDF', width = 400, height = 400)
    l = __L(x, epsilon, data)
    u = __U(x, epsilon, data) 
    p.circle(x, l, color = color)
    p.circle(x, u, color = color)
    # overlay with experimental ECDF
    iqplot.ecdf(data, p = p, conf_int = True)
    return p
File:      c:\users\rashi jeeda\documents\caltech\clas

In [23]:
p1 = mc.plot_conf_int(labeled, "Confidence Interval for Labeled", "Time to catastrophe (s)")
bokeh.io.show(p1)

In [30]:
p2 = mc.plot_conf_int(not_labeled, "Confidence Interval for Unlabeled", "Time to catastrophe (s)")
bokeh.io.show(p2)

In [28]:
c = row(p1, p2)

bokeh.io.show(c)

See help(function) for more information on the functions and their arguments.